<a href="https://colab.research.google.com/github/deepakri201/NLSTNatureSciData/blob/main/TechnicalValidation/consistencyChecks/NLSTSegVsTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare NLSTSeg lung lobe location to TotalSegmentator

This notebook compare the lung lobe assignment of the lesions to what is generated by TotalSegmentator.


Deepa Krishnaswamy

Brigham and Women's Hospital

December 2025

# Parameterization

In [1]:
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# Environment setup

In [2]:
!pip install pydicom
import pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.3 MB/s eta 0:00:00


In [3]:
!pip install SimpleITK
import SimpleITK as sitk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 13.7 MB/s eta 0:00:00


In [4]:
# DCMQI - to convert nifti to DICOM Segmentation object
!wget https://github.com/QIICR/dcmqi/releases/download/v1.4.0/dcmqi-1.4.0-linux.tar.gz
!tar zxvf dcmqi-1.4.0-linux.tar.gz
!cp dcmqi-1.4.0-linux/bin/* /usr/local/bin/

--2025-12-22 19:53:38--  https://github.com/QIICR/dcmqi/releases/download/v1.4.0/dcmqi-1.4.0-linux.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/50675718/915cb14a-48f5-4288-9a5f-bfbcc67daf43?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-22T20%3A38%3A58Z&rscd=attachment%3B+filename%3Ddcmqi-1.4.0-linux.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-22T19%3A38%3A38Z&ske=2025-12-22T20%3A38%3A58Z&sks=b&skv=2018-11-09&sig=RAuNByCB1s7gwamZFnjOuM5IvimEVpVRcYY40QAuh00%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NjQzNTAxOSwibmJmIjoxNzY2NDMzMjE5LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjd

In [5]:
import pandas as pd
import json
import nibabel as nib
import numpy as np

from google.cloud import bigquery
from google.cloud import storage

import time
import shutil

In [6]:
!apt-get -qq install aria2

Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.11) ...
/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a

# Functions

In [7]:
def resample_image_or_mask(fixed_image_filename, moving_image_filename, warped_filename, warp_mask=0):
  """ This function resamples an image or a mask using SimpleITK.
      Inputs:
        fixed_image_filename  - the fixed image filename
        moving_image_filename - the moving image filename
        warped_filename       - the warped image/mask filename that will be saved to disk
        warp_mask             - if 0 - warp image (linear interpolation), if 1 - warp mask (nearest neighbor)
      Outputs:
        writes the warped image/mask to disk
  """

  fixed_itk_image = sitk.ReadImage(fixed_image_filename)
  moving_itk_image = sitk.ReadImage(moving_image_filename)
  # moving_itk_image = sitk.Cast(moving_itk_image, sitk.sitkFloat32)

  print('moving image type: ' + str(moving_itk_image.GetPixelIDTypeAsString()))

  original_spacing = moving_itk_image.GetSpacing()
  original_size = moving_itk_image.GetSize()
  out_spacing = fixed_itk_image.GetSpacing()
  out_size = fixed_itk_image.GetSize()

  print("Fixed origin:", fixed_itk_image.GetOrigin())
  print("Fixed spacing:", fixed_itk_image.GetSpacing())
  print("Fixed direction:", fixed_itk_image.GetDirection())

  print("Moving origin:", moving_itk_image.GetOrigin())
  print("Moving spacing:", moving_itk_image.GetSpacing())
  print("Moving direction:", moving_itk_image.GetDirection())

  resample = sitk.ResampleImageFilter()
  resample.SetOutputSpacing(out_spacing)
  resample.SetSize(out_size)
  resample.SetOutputDirection(fixed_itk_image.GetDirection())
  resample.SetOutputOrigin(fixed_itk_image.GetOrigin())
  resample.SetTransform(sitk.Transform())
  # resample.SetDefaultPixelValue(T2_itk_image.GetPixelIDValue())
  resample.SetDefaultPixelValue(0)

  # if equal to 1, warp using nearest neighbor = ROI
  if (warp_mask):
    resample.SetInterpolator(sitk.sitkNearestNeighbor)
  else:
    resample.SetInterpolator(sitk.sitkLinear)

  result_image = resample.Execute(moving_itk_image)
  sitk.WriteImage(result_image, warped_filename)

  print('result image type: ' + str(result_image.GetPixelIDTypeAsString()))

  return

# Download CT data from Zenodo

We need the original CT data from Zenodo. Since this was used to create our DICOM SEG files, and therefore the orientation matches.

But beware! Downloading from Zenodo can take a long time depending on your internet connection.

In [8]:
start_time = time.time()
!aria2c -x 16 -s 16 -k 1M "https://zenodo.org/records/14838349/files/2_LungTumor.zip?download=1"
end_time = time.time()
print('2_LungTumor.zip: ' + str((end_time-start_time)/60))

start_time = time.time()
!aria2c -x 16 -s 16 -k 1M "https://zenodo.org/records/14838349/files/3_LungTumor.zip?download=1"
end_time = time.time()
print('3_LungTumor.zip: ' + str((end_time-start_time)/60))

start_time = time.time()
!aria2c -x 16 -s 16 -k 1M "https://zenodo.org/records/14838349/files/4_LungTumor.zip?download=1"
end_time = time.time()
print('4_LungTumor.zip: ' + str((end_time-start_time)/60))

start_time = time.time()
!aria2c -x 16 -s 16 -k 1M "https://zenodo.org/records/14838349/files/5_LungTumor.zip?download=1"
end_time = time.time()
print('5_LungTumor.zip: ' + str((end_time-start_time)/60))

start_time = time.time()
!aria2c -x 16 -s 16 -k 1M "https://zenodo.org/records/14838349/files/6_LungTumor.zip?download=1"
end_time = time.time()
print('6_LungTumor.zip: ' + str((end_time-start_time)/60))

start_time = time.time()
!aria2c -x 16 -s 16 -k 1M "https://zenodo.org/records/14838349/files/7_LungTumor.zip?download=1"
end_time = time.time()
print('7_LungTumor.zip: ' + str((end_time-start_time)/60))


12/16 21:41:20 [NOTICE] Downloading 1 item(s)
 *** Download Progress Summary as of Tue Dec 16 21:42:21 2025 *** 
=
[#b57f01 4.0GiB/4.5GiB(90%) CN:16 DL:62MiB ETA:7s]
FILE: /content/2_LungTumor.zip
-


12/16 21:42:27 [NOTICE] Download complete: /content/2_LungTumor.zip

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b57f01|OK  |    69MiB/s|/content/2_LungTumor.zip

Status Legend:
(OK):download completed.
2_LungTumor.zip: 1.1196715513865152

12/16 21:42:28 [NOTICE] Downloading 1 item(s)
 *** Download Progress Summary as of Tue Dec 16 21:43:28 2025 *** 
=
[#c9b78f 1.9GiB/4.4GiB(43%) CN:16 DL:30MiB ETA:1m24s]
FILE: /content/3_LungTumor.zip
-


12/16 21:44:25 [NOTICE] Download complete: /content/3_LungTumor.zip

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c9b78f|OK  |    38MiB/s|/content/3_LungTumor.zip

Status Legend:
(O

In [9]:
%%capture
!unzip "/content/2_LungTumor.zip"
!unzip "/content/3_LungTumor.zip"
!unzip "/content/4_LungTumor.zip"
!unzip "/content/5_LungTumor.zip"
!unzip "/content/6_LungTumor.zip"
!unzip "/content/7_LungTumor.zip"

# Compare

## Query to get intersecting SeriesInstanceUIDs

In [10]:
# First we find the ReferencedSeriesInstanceUIDs that overlap between
# NLSTSeg lesions and TotalSegmentator SEG files

client_bq = bigquery.Client(project=project_name)
query = f"""
      WITH nlstseg AS (
        SELECT DISTINCT
          PatientID,
          StudyInstanceUID,
          SeriesInstanceUID,
          ReferencedSeriesSequence[SAFE_OFFSET(0)].SeriesInstanceUID AS ReferencedSeriesInstanceUID,
          gcs_url
        FROM
          `bigquery-public-data.idc_v23.dicom_all`
        WHERE
          analysis_result_id = 'NLSTSeg' AND
          Modality = 'SEG'
      )
      SELECT DISTINCT
        dicom_all.PatientID,
        dicom_all.StudyInstanceUID,
        dicom_all.SeriesInstanceUID AS TS_SeriesInstanceUID,
        nlstseg.SeriesInstanceUID AS NLSTSeg_SeriesInstanceUID,
        nlstseg.ReferencedSeriesInstanceUID AS CT_SeriesInstanceUID,
        dicom_all.gcs_url AS TS_gcs_url,
        nlstseg.gcs_url AS NLSTSeg_gcs_url
      FROM
        `bigquery-public-data.idc_v23.dicom_all` as dicom_all
      JOIN
        nlstseg
      ON
        nlstseg.ReferencedSeriesInstanceUID = dicom_all.ReferencedSeriesSequence[SAFE_OFFSET(0)].SeriesInstanceUID
      WHERE
        analysis_result_id = 'TotalSegmentator-CT-Segmentations' AND
        Modality = 'SEG'
      ORDER BY
        dicom_all.PatientID,
        dicom_all.StudyInstanceUID
      """
df_seg = client_bq.query(query).to_dataframe()

In [11]:
df_seg.head()

,PatientID,StudyInstanceUID,TS_SeriesInstanceUID,NLSTSeg_SeriesInstanceUID,CT_SeriesInstanceUID,TS_gcs_url,NLSTSeg_gcs_url
0,100012,1.2.840.113654.2.55.38321092839390108338558865...,1.2.276.0.7230010.3.1.3.313263360.36578.170632...,1.2.276.0.7230010.3.1.3.481037312.8266.1761239...,1.2.840.113654.2.55.13508825378604927579146345...,gs://idc-open-data/efddd212-ecda-4704-bc04-319...,gs://idc-open-data/816bef13-9926-4031-8a23-8a0...
1,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.276.0.7230010.3.1.3.313263360.138.17063254...,1.2.276.0.7230010.3.1.3.481037312.9241.1761239...,1.2.840.113654.2.55.15708941008648745210499888...,gs://idc-open-data/ca6162dd-c7bb-4229-af81-f0e...,gs://idc-open-data/1b99cce0-9836-460d-9a9a-4a4...
2,100158,1.2.840.113654.2.55.81185422866512279860334872...,1.2.276.0.7230010.3.1.3.313263360.8087.1706320...,1.2.276.0.7230010.3.1.3.481037312.10206.176123...,1.2.840.113654.2.55.31060976780967844152296392...,gs://idc-open-data/a32b0f7b-79d1-42e2-b0bf-c2d...,gs://idc-open-data/ac86b50d-429e-4b80-b63d-9f9...
3,100242,1.2.840.113654.2.55.22835224307907880875083018...,1.2.276.0.7230010.3.1.3.313263360.7940.1706324...,1.2.276.0.7230010.3.1.3.481037312.11172.176123...,1.2.840.113654.2.55.38995485391900019876570761...,gs://idc-open-data/ba6d6751-4deb-4696-a6fb-3c0...,gs://idc-open-data/0e95deee-ab56-4ec3-bb62-b58...
4,100280,1.2.840.113654.2.55.30207427775351702734806349...,1.2.276.0.7230010.3.1.3.313263360.7534.1706323...,1.2.276.0.7230010.3.1.3.481037312.12152.176123...,1.2.840.113654.2.55.11168439182281552750699540...,gs://idc-open-data/5438e45c-4aa8-4295-ab09-eba...,gs://idc-open-data/fbb7a8de-6ad6-40aa-9684-cd8...


In [12]:
# Print some metrics

print(len(df_seg))
print(len(list(set(df_seg['PatientID'].values))))
print(len(list(set(df_seg['StudyInstanceUID'].values))))

575
575
575


In [13]:
# Add the filenames for the CT files to the dataframe

PatientID_list_2 = os.listdir("/content/NLSTseg_2_LungTumor")
PatientID_list_3 = os.listdir("/content/NLSTseg_3_LungTumor")
PatientID_list_4 = os.listdir("/content/NLSTseg_4_LungTumor")
PatientID_list_5 = os.listdir("/content/NLSTseg_5_LungTumor")
PatientID_list_6 = os.listdir("/content/NLSTseg_6_LungTumor")
PatientID_list_7 = os.listdir("/content/NLSTseg_7_LungTumor")

CT_filenames = []
for n in range(0,len(df_seg)):
  PatientID = df_seg['PatientID'].values[n]
  # get the correct directory that this patient is in
  if PatientID in PatientID_list_2:
    CT_filename = os.path.join("/content/NLSTseg_2_LungTumor", PatientID, PatientID + "_CT.nii.gz")
    CT_filenames.append(CT_filename)
  elif PatientID in PatientID_list_3:
    CT_filename = os.path.join("/content/NLSTseg_3_LungTumor", PatientID, PatientID + "_CT.nii.gz")
    CT_filenames.append(CT_filename)
  elif PatientID in PatientID_list_4:
    CT_filename = os.path.join("/content/NLSTseg_4_LungTumor", PatientID, PatientID + "_CT.nii.gz")
    CT_filenames.append(CT_filename)
  elif PatientID in PatientID_list_5:
    CT_filename = os.path.join("/content/NLSTseg_5_LungTumor", PatientID, PatientID + "_CT.nii.gz")
    CT_filenames.append(CT_filename)
  elif PatientID in PatientID_list_6:
    CT_filename = os.path.join("/content/NLSTseg_6_LungTumor", PatientID, PatientID + "_CT.nii.gz")
    CT_filenames.append(CT_filename)
  elif PatientID in PatientID_list_7:
    CT_filename = os.path.join("/content/NLSTseg_7_LungTumor", PatientID, PatientID + "_CT.nii.gz")
    CT_filenames.append(CT_filename)
  else:
    print('ERROR: PatientID not found: ' + str(PatientID))
    continue

df_seg['NLSTSeg_CT_path'] = CT_filenames


# Perform the comparison

In [14]:
df_seg.head()

,PatientID,StudyInstanceUID,TS_SeriesInstanceUID,NLSTSeg_SeriesInstanceUID,CT_SeriesInstanceUID,TS_gcs_url,NLSTSeg_gcs_url,NLSTSeg_CT_path
0,100012,1.2.840.113654.2.55.38321092839390108338558865...,1.2.276.0.7230010.3.1.3.313263360.36578.170632...,1.2.276.0.7230010.3.1.3.481037312.8266.1761239...,1.2.840.113654.2.55.13508825378604927579146345...,gs://idc-open-data/efddd212-ecda-4704-bc04-319...,gs://idc-open-data/816bef13-9926-4031-8a23-8a0...,/content/NLSTseg_2_LungTumor/100012/100012_CT....
1,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.276.0.7230010.3.1.3.313263360.138.17063254...,1.2.276.0.7230010.3.1.3.481037312.9241.1761239...,1.2.840.113654.2.55.15708941008648745210499888...,gs://idc-open-data/ca6162dd-c7bb-4229-af81-f0e...,gs://idc-open-data/1b99cce0-9836-460d-9a9a-4a4...,/content/NLSTseg_2_LungTumor/100147/100147_CT....
2,100158,1.2.840.113654.2.55.81185422866512279860334872...,1.2.276.0.7230010.3.1.3.313263360.8087.1706320...,1.2.276.0.7230010.3.1.3.481037312.10206.176123...,1.2.840.113654.2.55.31060976780967844152296392...,gs://idc-open-data/a32b0f7b-79d1-42e2-b0bf-c2d...,gs://idc-open-data/ac86b50d-429e-4b80-b63d-9f9...,/content/NLSTseg_2_LungTumor/100158/100158_CT....
3,100242,1.2.840.113654.2.55.22835224307907880875083018...,1.2.276.0.7230010.3.1.3.313263360.7940.1706324...,1.2.276.0.7230010.3.1.3.481037312.11172.176123...,1.2.840.113654.2.55.38995485391900019876570761...,gs://idc-open-data/ba6d6751-4deb-4696-a6fb-3c0...,gs://idc-open-data/0e95deee-ab56-4ec3-bb62-b58...,/content/NLSTseg_2_LungTumor/100242/100242_CT....
4,100280,1.2.840.113654.2.55.30207427775351702734806349...,1.2.276.0.7230010.3.1.3.313263360.7534.1706323...,1.2.276.0.7230010.3.1.3.481037312.12152.176123...,1.2.840.113654.2.55.11168439182281552750699540...,gs://idc-open-data/5438e45c-4aa8-4295-ab09-eba...,gs://idc-open-data/fbb7a8de-6ad6-40aa-9684-cd8...,/content/NLSTseg_2_LungTumor/100280/100280_CT....


In [15]:
if not os.path.isdir("/content/lesion_matching"):
  os.makedirs("/content/lesion_matching", exist_ok=True)

CT_SeriesInstanceUID_list = list(set(df_seg['CT_SeriesInstanceUID'].values))
print('Num series: ' + str(len(CT_SeriesInstanceUID_list)))

checkpoints = {int(len(CT_SeriesInstanceUID_list) * i / 10) for i in range(1, 11)}

Num series: 575


In [17]:
series_index

57

In [20]:
for series_index,CT_SeriesInstanceUID in enumerate(CT_SeriesInstanceUID_list[56:],1):

  print('***** series_index: ' + str(series_index) + ' series: ' + str(CT_SeriesInstanceUID) + ' *****')

  ##########################
  ### Download the files ###
  ##########################

  # Get paths and UIDs
  TS_SeriesInstanceUID = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['TS_SeriesInstanceUID'].values[0]
  NLSTSeg_SeriesInstanceUID = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['NLSTSeg_SeriesInstanceUID'].values[0]
  TS_path = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['TS_gcs_url'].values[0]
  NLSTSeg_path = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['NLSTSeg_gcs_url'].values[0]
  NLSTSeg_CT_path = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['NLSTSeg_CT_path'].values[0]
  # download the two SEG files
  !gsutil cp $TS_path "/content/ts.dcm"
  !gsutil cp $NLSTSeg_path "/content/nlstseg.dcm"
  # copy the CT file
  shutil.copy(NLSTSeg_CT_path, "/content/nlstseg_ct.nii.gz")

  # Other UIDs
  PatientID = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['PatientID'].values[0]
  StudyInstanceUID = df_seg[df_seg['CT_SeriesInstanceUID']==CT_SeriesInstanceUID]['StudyInstanceUID'].values[0]

  #####################################
  ### Convert from DICOM SEG to nii ###
  #####################################

  # NLSTSeg
  if not os.path.isdir("/content/nlstseg_output"):
    os.makedirs("/content/nlstseg_output", exist_ok=True)
  !segimage2itkimage -t "nii" --outputDirectory "/content/nlstseg_output" --inputDICOM "/content/nlstseg.dcm"  --mergeSegments
  json_filename = "/content/nlstseg_output/meta.json"
  with open(json_filename, 'r') as file:
    nlstseg_data = json.load(file)
  nlstseg_data_pretty = json.dumps(nlstseg_data, indent=2)
  # TS
  if not os.path.isdir("/content/ts_output"):
    os.makedirs("/content/ts_output", exist_ok=True)
  !segimage2itkimage -t "nii" --outputDirectory "/content/ts_output" --inputDICOM "/content/ts.dcm"  --mergeSegments
  # Let's print what's in the json file
  json_filename = "/content/ts_output/meta.json"
  with open(json_filename, 'r') as file:
    ts_data = json.load(file)
  ts_data_pretty = json.dumps(ts_data, indent=2)

  #############################################
  ### Resample both to the original CT file ###
  #############################################

  # This is the one from NLSTSeg - not the same as taking DICOM and converting to CT using dcm2niix - which was what was used for TS
  # NLSTSeg
  fixed_image_filename = '/content/nlstseg_ct.nii.gz'
  moving_image_filename = '/content/nlstseg_output/1.nii.gz'
  warped_filename = '/content/nlstseg_warped.nii.gz'
  warp_mask = 1
  resample_image_or_mask(fixed_image_filename, moving_image_filename, warped_filename, warp_mask)
  # TS
  fixed_image_filename = '/content/nlstseg_ct.nii.gz'
  moving_image_filename = '/content/ts_output/1.nii.gz'
  warped_filename = '/content/ts_warped.nii.gz'
  warp_mask = 1
  resample_image_or_mask(fixed_image_filename, moving_image_filename, warped_filename, warp_mask)
  # Load the resampled files
  ts_nii = "/content/ts_warped.nii.gz"
  nlstseg_nii = "/content/nlstseg_warped.nii.gz"
  ts_img = nib.load(ts_nii).get_fdata()
  nlstseg_img = nib.load(nlstseg_nii).get_fdata()
  print("ts_img: " + str(ts_img.shape))
  print("nlstseg_img: " + str(nlstseg_img.shape))

  # ### Get corresponding info from df_nlstseg_all ###
  # df_lesion_info = df_nlstseg_all[df_nlstseg_all['segmented_SeriesInstanceUID']==segmented_SeriesInstanceUID]
  # Form this df_lesion_info from the json file instead.. since we anyway need to download it to compute the overlap
  with open("/content/nlstseg_output/meta.json", "r") as f:
    seg_json = json.load(f)
  df_top = pd.DataFrame([seg_json])
  df_lesion_info = (
      df_top
      .explode("segmentAttributes")
      .explode("segmentAttributes")
  )
  # Normalize segment dicts automatically
  df_lesion_info = pd.json_normalize(
      df_lesion_info["segmentAttributes"]
  )
  num_lesions = len(df_lesion_info)
  print('num_lesions: ' + str(num_lesions))

  ############################
  ### Create some mappings ###
  ############################

  # AnatomicRegion in NLSTSeg to SegmentLabel in TS
  mapping_nlstseg_to_ts = dict()
  mapping_nlstseg_to_ts = {"Upper lobe of right lung": "Right Upper lobe of lung",
                           "Upper lobe of left lung": "Left Upper lobe of lung",
                           "Lower lobe of right lung": "Right Lower lobe of lung",
                           "Lower lobe of left lung": "Left Lower lobe of lung",
                           "Middle lobe of right lung": "Middle lobe of right lung"}
  # Simplified map to binary
  mapping_ts_lung_to_label = dict()
  ### Instead we use all from the json file - later I could get this from the segmentations big query table. ###
  # assuming `data` is your JSON dict
  segments = [s for sublist in ts_data["segmentAttributes"] for s in sublist]
  # use json_normalize to flatten nested fields automatically
  df = pd.json_normalize(
      segments,
      sep='_'
  )
  # now add top-level metadata to every row
  meta_fields = [
      "BodyPartExamined",
      "ClinicalTrialCoordinatingCenterName",
      "ClinicalTrialSeriesID",
      "ClinicalTrialTimePointID",
      "ContentCreatorName",
      "InstanceNumber",
      "SeriesDescription",
      "SeriesNumber"
  ]
  for field in meta_fields:
      df[field] = ts_data.get(field)
  # Now form the dict
  keys = df['SegmentLabel'].values
  values = df['labelID'].values
  mapping_ts_lung_to_label = dict(zip(keys, values))
  # Now do a reverse mapping
  mapping_label_to_ts_lung = {v: k for k, v in mapping_ts_lung_to_label.items()}

  ### For each lesion, get the lobe that it's in from TS seg ###

  df_ts_regions_summary = pd.DataFrame()

  for lesion_index in range(0,num_lesions):

    # For the lesion index, find the ids to compare
    # NLSTSeg_AnatomicRegion = df_lesion_info['AnatomicRegion'].values[lesion_index]
    NLSTSeg_AnatomicRegion = df_lesion_info['AnatomicRegionSequence.CodeMeaning'].values[lesion_index]
    # NLSTSeg_SegmentNumber = df_lesion_info['SegmentNumber'].values[lesion_index]
    NLSTSeg_SegmentNumber = df_lesion_info['labelID'].values[lesion_index]

    if NLSTSeg_AnatomicRegion not in list(mapping_nlstseg_to_ts.keys()):

      print(f"Region '{NLSTSeg_AnatomicRegion}' does not exist in TS — recording as unmatched.")
      df_temp = pd.DataFrame({
          "TS_Segment": ["No corresponding region in TS"],
          "Percentage": [0],
          "NLSTSeg_Segment": [NLSTSeg_AnatomicRegion],
          "Lesion": [df_lesion_info['SegmentLabel'].values[lesion_index]],
          "PatientID": [PatientID],
          "StudyInstanceUID": [StudyInstanceUID],
          "segmented_SeriesInstanceUID": [CT_SeriesInstanceUID],
          "TS_SeriesInstanceUID": [TS_SeriesInstanceUID],
          "NLSTSeg_SeriesInstanceUID": [NLSTSeg_SeriesInstanceUID],
      })
      df_ts_regions_summary = pd.concat([df_ts_regions_summary, df_temp])

    else:

      # Get the lesion_index values
      nlstseg_lesion_indices = np.where(nlstseg_img==NLSTSeg_SegmentNumber)

      # Get the values in the TS segmentation
      ts_values = ts_img[nlstseg_lesion_indices]
      # remove 0's from array - background
      ts_values = ts_values[ts_values != 0]

      # Now find the lung regions that correspond to the ts_values
      ts_regions = [mapping_label_to_ts_lung[f] for f in ts_values]

      # Now create df
      df_ts_regions = pd.DataFrame(ts_regions, columns=['TS_Segment'])
      # Compute counts and percentages
      df_temp = (
          df_ts_regions['TS_Segment']
          .value_counts(normalize=True)  # gives fraction
          .mul(100)                      # convert to percentage
          .rename('Percentage')
          .reset_index()
          .rename(columns={'index': 'TS_Segment'})
      )
      num_overlapping_regions = len(df_temp)

      # for each overlapping region
      for region in range(0,num_overlapping_regions):
        df_temp['NLSTSeg_Segment'] = mapping_nlstseg_to_ts[NLSTSeg_AnatomicRegion]
        df_temp['Lesion'] = df_lesion_info['SegmentLabel'].values[lesion_index]
        df_temp['PatientID'] = PatientID
        df_temp['StudyInstanceUID'] = StudyInstanceUID
        df_temp['segmented_SeriesInstanceUID'] = CT_SeriesInstanceUID
        df_temp['TS_SeriesInstanceUID'] = TS_SeriesInstanceUID
        df_temp['NLSTSeg_SeriesInstanceUID'] = NLSTSeg_SeriesInstanceUID
        df_ts_regions_summary = pd.concat([df_ts_regions_summary,df_temp])

  ###############################
  ### Add in matching columns ###
  ###############################

  # Add a matching column - on a per lesion basis
  try:
    df_ts_regions_summary["per_lesion_match"] = (df_ts_regions_summary["NLSTSeg_Segment"] == df_ts_regions_summary["TS_Segment"]).astype(int)
  except:
    print('ERROR: cannot find one of the segments in')
    continue
  # Add a matching column - on a per series basis
  # If any of the per_lesion_match in a series are 0, set to 0. else if all 1 set to 1.
  df_ts_regions_summary["per_series_match"] = (
      df_ts_regions_summary.groupby(["PatientID", "StudyInstanceUID", "segmented_SeriesInstanceUID"])["per_lesion_match"]
        .transform("min")
  )
  # Add in the viewer url
  # Holds the segmented_SeriesInstanceUID, TS_SeriesInstanceUID and the NLSTSeg_SeriesInstanceUID
  viewer_url_list = ["https://viewer.imaging.datacommons.cancer.gov/v3/viewer/?StudyInstanceUIDs=" +
                    str(study) +  "&SeriesInstanceUIDs=" + str(seg_series) + ',' + str(ts_series) + ',' + str(nlstseg_series)
                    for study,seg_series,ts_series,nlstseg_series in
                    zip(df_ts_regions_summary['StudyInstanceUID'].values,
                        df_ts_regions_summary['segmented_SeriesInstanceUID'].values,
                        df_ts_regions_summary['TS_SeriesInstanceUID'].values,
                        df_ts_regions_summary['NLSTSeg_SeriesInstanceUID'].values)]

  df_ts_regions_summary['viewer_url'] = viewer_url_list

  # Reorder columns
  df_ts_regions_summary = df_ts_regions_summary[['PatientID', 'StudyInstanceUID', 'segmented_SeriesInstanceUID',
                                                'TS_SeriesInstanceUID', 'NLSTSeg_SeriesInstanceUID',
                                                'Lesion', 'NLSTSeg_Segment', 'TS_Segment', 'Percentage',
                                                'per_lesion_match', 'per_series_match',
                                                'viewer_url']]

  # Save this csv and copy to a bucket
  df_ts_regions_summary_filename = os.path.join("/content/lesion_matching", str(CT_SeriesInstanceUID) + ".csv")
  df_ts_regions_summary.to_csv(df_ts_regions_summary_filename)
  ### DELETE LATER ###
  bucket_filename = os.path.join("gs://nlstseg_seg_and_sr_analysis/lesion_matching2", str(CT_SeriesInstanceUID) + ".csv")
  !gsutil cp $df_ts_regions_summary_filename $bucket_filename

  ### Delete files ###
  # NLSTSeg files
  !rm "/content/nlstseg.dcm"
  !rm "/content/nlstseg_warped.nii.gz"
  !rm "/content/nlstseg_output/1.nii.gz"
  !rm "/content/nlstseg_output/meta.json"
  # TS files
  !rm "/content/ts.dcm"
  !rm "/content/ts_warped.nii.gz"
  !rm "/content/ts_output/1.nii.gz"
  !rm "/content/ts_output/meta.json"
  # Other files
  !rm "/content/nlstseg_ct.nii.gz"

  if series_index in checkpoints:
    print(f"{(series_index / len(CT_SeriesInstanceUID_list)) * 100:.0f}% of series processed.")


Streaming output truncated to the last 5000 lines.
Image Orientation Patient set to : 1, 0, 0, 0, 1, 0
Identified 1 groups of non-overlapping segments
Writing itk image to /content/nlstseg_output/1.nii.gz ... done
dcmqi repository URL: https://github.com/QIICR/dcmqi revision: 4e5b700 tag: v1.4.0
Loading DICOM SEG file /content/ts.dcm
Row direction: 1 0 0
Col direction: 0 -1 0
Z direction: 0 0 -1
Total frames: 2909
Total frames with unique IPP: 137
Total overlapping frames: 137
Origin: [-134.1, 179.837, 1.91]
Slice extent: 272
Slice spacing: 2
Image Orientation Patient set to : 1, 0, 0, 0, -1, 0
Identified 1 groups of non-overlapping segments
Writing itk image to /content/ts_output/1.nii.gz ... done
moving image type: 16-bit signed integer
Fixed origin: (-134.10000610351562, -107.5999984741211, -270.0899963378906)
Fixed spacing: (0.5625, 0.5625, 2.0)
Fixed direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
Moving origin: (-134.10000610351562, -107.5999984741211, -68.08999633789062

In [19]:
series_index

57

# Merge the comparison csvs and download

In [ ]:
# Zip the lesion matching csvs

!zip -r lesion_matching.zip "/content/lesion_matching"

In [15]:
# Merge the csvs
# Create df and save as csv

df_merged = pd.DataFrame()
filenames = os.listdir("/content/lesion_matching2")
filenames = [os.path.join("/content/lesion_matching2", f) for f in filenames]
num_files = len(filenames)

print(filenames)
print(num_files)

for f in range(0,num_files):
  df_temp = pd.read_csv(filenames[f])
  df_merged = pd.concat([df_merged, df_temp])

df_merged.to_csv("/content/lesion_matching.csv")

['/content/lesion_matching2/1.3.6.1.4.1.14519.5.2.1.7009.9004.113963576659208997922505947759.csv', '/content/lesion_matching2/1.3.6.1.4.1.14519.5.2.1.7009.9004.279815614829054107333130436699.csv', '/content/lesion_matching2/1.2.840.113654.2.55.202590990749233587270867529690119914374.csv', '/content/lesion_matching2/1.3.6.1.4.1.14519.5.2.1.7009.9004.732617958380743426518581294837.csv', '/content/lesion_matching2/1.2.840.113654.2.55.169697003742573435409614923241347675640.csv', '/content/lesion_matching2/1.2.840.113654.2.55.171251717048735953113234672113612055668.csv', '/content/lesion_matching2/1.2.840.113654.2.55.297475359465669712309938774091708798816.csv', '/content/lesion_matching2/1.2.840.113654.2.55.135968128742262057179054957067531073282.csv', '/content/lesion_matching2/1.3.6.1.4.1.14519.5.2.1.7009.9004.247714574186092243007275105443.csv', '/content/lesion_matching2/1.2.840.113654.2.55.156187021228815051225055591473745716524.csv', '/content/lesion_matching2/1.3.6.1.4.1.14519.5.2.